중간발표 이후 체계적인 학습을 위한 전처리


1. Get Pre-trained Model --->  model = YOLO('yolov8n.pt')'
        
2. 단일 식자제에 대해서 모델 학습  
   36 Label 55,000 Dataset / 11 Label 13,000 Dataset  
   복합 식자재셋 라벨에 맞게 정리 후 병합  
     
   Data_Food_Ingredients_0803  (36, 55,000)  
   Data_Food_Ingredients_0807_v3  (11, 13,000)
   
     
     
3. 복합 식자재에 대해서 모델 학습  
   37 Label 12,000 Dataset 에 대해서 모델을 추가 학습
     
   fused_food_ingredients_0806  (37, 12,000)

# Preparation

In [ ]:
!git clone https://github.com/ultralytics/ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 35864, done.
remote: Counting objects: 100% (4441/4441), done.
remote: Compressing objects: 100% (1957/1957), done.
remote: Total 35864 (delta 3067), reused 3661 (delta 2463), pack-reused 31423
Receiving objects: 100% (35864/35864), 29.86 MiB | 9.42 MiB/s, done.
Resolving deltas: 100% (25826/25826), done.


In [ ]:
%cd ./ultralytics

/content/ultralytics


In [ ]:
!pip install -e .

In [ ]:
!pip install -q roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 486.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import torch
import yaml
from IPython.display import Image, clear_output  # to display images

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
data_yaml_36_55000 = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0803/data.yaml'
data_yaml_11_13000 = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0807_v3/data.yaml'
data_yaml_37_12000 = '/content/drive/MyDrive/DAB/fused_food_ingredients_0806/data.yaml'

### Function for counting files

In [ ]:
def count_files_in_dataset(train_path, valid_path, test_path, dataset_name):
    """
    주어진 경로의 데이터셋에서 파일 수를 계산하여 출력합니다.

    Args:
        train_path (str): 훈련 데이터셋 경로
        valid_path (str): 검증 데이터셋 경로
        test_path (str): 테스트 데이터셋 경로
        dataset_name (str): 데이터셋 이름
    """
    train_files = len(os.listdir(train_path))
    valid_files = len(os.listdir(valid_path))
    test_files = len(os.listdir(test_path))

    total_files = train_files + valid_files + test_files
    print(f'{dataset_name} - 총 파일 개수: {total_files}')

# 첫 번째 데이터셋
def dataset_0803():
    labels_path_train = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0803/train/images'
    labels_path_valid = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0803/valid/images'
    labels_path_test = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0803/test/images'

    count_files_in_dataset(labels_path_train, labels_path_valid, labels_path_test, 'Data_Food_Ingredients_0803')

# 두 번째 데이터셋
def dataset_0807_v3():
    labels_path_train = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0807_v3/train/images'
    labels_path_valid = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0807_v3/valid/images'
    labels_path_test = '/content/drive/MyDrive/DAB/Data_Food_Ingredients_0807_v3/test/images'

    count_files_in_dataset(labels_path_train, labels_path_valid, labels_path_test, 'Data_Food_Ingredients_0807_v3')

# 세 번째 데이터셋
def fused_food_ingredients_0806():
    labels_path_train = '/content/drive/MyDrive/DAB/fused_food_ingredients_0806/train/images'
    labels_path_valid = '/content/drive/MyDrive/DAB/fused_food_ingredients_0806/valid/images'
    labels_path_test = '/content/drive/MyDrive/DAB/fused_food_ingredients_0806/test/images'

    count_files_in_dataset(labels_path_train, labels_path_valid, labels_path_test, 'Fused_Food_Ingredients_0806')

### 36 Label 55,198 Dataset (Single)

In [ ]:
with open(data_yaml_36_55000) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

{'names': ['beaf',
  'bellpepper',
  'carrot',
  'chicken',
  'chili',
  'cucumber',
  'egg',
  'eggplant',
  'fish',
  'garlic',
  'green_onion',
  'kimchi',
  'koreancabbage',
  'lettuce',
  'mungbeansprout',
  'mushroom',
  'onion',
  'pork',
  'potato',
  'pumpkin',
  'radish',
  'sausage',
  'shrimp',
  'squid',
  'tofu',
  'tomato',
  'bab',
  'bean_sprouts',
  'bread',
  'cut_mackerel',
  'fishcake',
  'garlic_clove',
  'ginger',
  'mackerel',
  'perilla',
  'sweet_potato'],
 'nc': 36,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'food-ingredients-ftnpv',
  'url': 'https://universe.roboflow.com/data-orccx/food-ingredients-ftnpv/dataset/4',
  'version': 4,
  'workspace': 'data-orccx'},
 'test': '../test/images',
 'train': '../train/images',
 'val': '../valid/images'}

In [ ]:
dataset_0803()

Data_Food_Ingredients_0803 - 총 파일 개수: 55198


### 11 Label 13,478 Dataset (Single)

In [ ]:
with open(data_yaml_11_13000) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

{'train': '../train/images',
 'val': '../valid/images',
 'test': '../test/images',
 'nc': 11,
 'names': ['Enoki Mushroom',
  'Shiitake mushrooms',
  'apple',
  'banana',
  'brocoli',
  'cheese',
  'corn',
  'kimchi',
  'orange',
  'sausage',
  'tuna can'],
 'roboflow': {'workspace': 'd-ry6lg',
  'project': 'data_food_ingredients_0807',
  'version': 2,
  'license': 'CC BY 4.0',
  'url': 'https://universe.roboflow.com/d-ry6lg/data_food_ingredients_0807/dataset/2'}}

In [ ]:
dataset_0807_v3()

Data_Food_Ingredients_0807_v3 - 총 파일 개수: 13478


### 37 Label 11,945 Dataset (Fused)

In [ ]:
with open(data_yaml_37_12000) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

{'train': '../train/images',
 'val': '../valid/images',
 'test': '../test/images',
 'nc': 37,
 'names': ['Enoki Mushroom',
  'Shiitake mushrooms',
  'apple',
  'banana',
  'beaf',
  'bean_sprouts',
  'bellpepper',
  'bread',
  'brocoli',
  'cabbage',
  'carrot',
  'cheese',
  'chicken',
  'chili',
  'corn',
  'cucumber',
  'egg',
  'eggplant',
  'fish',
  'garlic',
  'ginger',
  'green_onion',
  'kimchi',
  'lettuce',
  'mushroom',
  'onion',
  'orange',
  'pork',
  'potato',
  'pumpkin',
  'radish',
  'sausage',
  'shrimp',
  'sweet_potato',
  'tofu',
  'tomato',
  'tuna can'],
 'roboflow': {'workspace': 'dab-zgcvo',
  'project': 'food_ingredients_fused',
  'version': 2,
  'license': 'CC BY 4.0',
  'url': 'https://universe.roboflow.com/dab-zgcvo/food_ingredients_fused/dataset/2'}}